# Smooth data
This is applied principaly to time series data

Content
- Promedio móvil simple
- Promedio móvil ponderado
- Promedio móvil exponencial

Sources

- Source dataset: https://www.tensorflow.org/tutorials/structured_data/time_series?hl=es-419

- Sorce dataset v2: https://keras.io/examples/timeseries/timeseries_weather_forecasting/
  
- Source lines charts: https://plotly.com/python/line-charts/

In [1]:
import os
# fix root path to save outputs
actual_path = os.path.abspath(os.getcwd())
list_root_path = actual_path.split('/')[:-1]
root_path = '/'.join(list_root_path)
os.chdir(root_path)
print('root path: ', root_path)

root path:  /Users/joseortega/Documents/GitHub/exploratory-data-analysis-ds


### 0. Package and load data
**Dataset: Tensorflow climate**

In [2]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

# plotly
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

In [3]:
# get data
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

# read data
data = pd.read_csv(csv_path)

# transform index into datetime format
index_datetime = pd.to_datetime(data.pop('Date Time'), format='%d.%m.%Y %H:%M:%S')

# set datetime as index
data.set_index(index_datetime, inplace =  True)

In [4]:
data.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
2009-01-01 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2009-01-01 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
2009-01-01 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
2009-01-01 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [5]:
data.shape

(420551, 14)

In [6]:
# sampling data to do calculations fast - weighted moving average has a "apply method" que hace lenta la ejecución
start_date = '2009-01-01 01:00:00'
end_date = '2009-02-01 00:00:00'

data_sampled = data.loc[start_date:end_date]

In [7]:
data_sampled.shape

(4459, 14)

## Concepts

- Definición **serie de tiempo**: secuencia de datos experimentales ordenados en el tiempo

- **El ordenamiento temporal da cabida a que exista correlación entre valores sucecivos** (el valor presente puede depender de los valores pasados)

- Lo que vuelve interesante **entender los mecanismos causales subyacentes detrás de un fenómeplo)

- Una serie de tiempo tiene diferentes componentes: **Tendencia, Estacionalidad, Ciclicidad y Residuo**. Tendencia: Incremento o decremento a largo plazo en los datos. No tiene que ser lineal, y puede cambiar de dirección

- Una de los componentes más útiles de predecir es la tendencia. **Una técnica para extraer la tendencia y eliminar ruido de los datos, muy simple y bastante efectiva es el suavizado**

## Auxiliar function to compare trends with data with different types of smooth

In [8]:
def plot_compare_tendencias(df_original, df_smoothed, kind_smooth, number_columns=2):
    '''plot_compare_tendencias
    Plot all the features in two differents dataframes in only one plot. The idea is compare the tendency of two differents dataframes where
    one dataframe is the original and the second is the dataframe with smoothed values

    Each feature is ploted in one subplot
    
    Args
        df_original (dataframe): original dataframe
        df_smoothed (dataframe): smoothed dataframe
        kind_smooth (string): kind of smooth. In the plot is showed only in the title
        number_columns (int): number of columns in the subplot. by default 2 columns
    '''
    # get list of features of both dataframes
    list_features = list(set(df_original.columns.tolist() + df_smoothed.columns.tolist()))

    # calculate number of rows (considering the number of colums passed as args)
    if (len(list_features) % number_columns) != 0:
        number_rows = (len(list_features) // number_columns) + 1
    else:
        number_rows = (len(list_features) // number_columns)

    # create fig to plot
    fig = make_subplots(rows=number_rows, cols=number_columns, subplot_titles=tuple(list_features))

    ########## for each feature plot:
    for index_feature in range(len(list_features)):
        feature = list_features[index_feature]

        # get indexes in the subplot (in plotly the indexes starts in 1)
        row = (index_feature // number_columns) + 1
        column = (index_feature % number_columns) + 1

        # plot feature of df_original in gray
        if feature in df_original.columns:
            fig.add_trace(
                go.Scatter(
                    x=df_original.index,
                    y=df_original[feature],
                    name='original - ' + feature,
                    line=dict(color='gray')
                ),
                row=row,
                col=column
            )

        # plot feature of df_smoothed in orange
        if feature in df_smoothed.columns:
            fig.add_trace(
                go.Scatter(
                    x=df_smoothed.index,
                    y=df_smoothed[feature],
                    name='df_smoothed - ' + feature,
                    line=dict(color='orange')
                ),
                row=row,
                col=column
            )

    # adjust the shape
    fig.update_layout(
        height = 350 * number_rows,  # largo
        width = 850 * number_columns,  # ancho
        title_text = f"Compare smooth data: {kind_smooth}",
        title_x=0.5,
        title_font = dict(size = 28)
    )

    return fig

## Promedio Móvil Simple
La manera más sencilla de suavizar es promediando

In [9]:
def apply_moving_average(df, window_size):
    """
    Moving average
    
    Args
        df (dataframe)
        window_size (int)
    
    Return
        df_smoothed (dataframe)
    """
    df_smoothed = df.rolling(window = window_size).mean()
    df_smoothed = df_smoothed.dropna()

    return df_smoothed

In [10]:
# apply moving average
window = 5
data_moving_average = apply_moving_average(data_sampled.copy(), window)

In [11]:
fig_moving_average = plot_compare_tendencias(df_original = data_sampled, 
                                             df_smoothed = data_moving_average,
                                             kind_smooth = 'moving average - window 5'
                                            )

# save
fig_moving_average.write_html("output_eda/2_univariate_analysis/fig_moving_average.html")

## Promedio Móvil Ponderado
- El promedio móvil simple se puede entender como un promedio en el que se da el mismo peso a los N­1 valores anteriores.

- Una aplicación común es darle pesos decrecientes a los valores entre más lejos estén en el pasado.

- **Esto permite que el promedio móvil responda más rápido en cambios súbitos de la serie**

- Cómo calcular promedio móvil pesado:

  1.- Define el tamaño de la ventana de tiempo (número de periodos) sobre el cual deseas calcular el promedio móvil ponderado.

  2.- Asigna pesos a cada uno de los valores en la ventana de tiempo. Los pesos pueden sumar 1 o cualquier otro valor deseado. Los pesos pueden ser proporcionales al tiempo, es decir, los valores más recientes pueden tener pesos más altos.

  3.- Multiplica cada valor en la ventana de tiempo por su respectivo peso.

  4.- Suma todos los valores ponderados.

  5.- Divide la suma de los valores ponderados entre la suma de los pesos para obtener el promedio móvil ponderado.

In [12]:
def apply_weighted_moving_average(df, weights):
    '''
    Calcula el promedio móvil ponderado de una serie de datos. "rolling" junto con "dot" para realizar la multiplicación y la suma ponderada. 
    
    Args
        df (dataframe)
        weights (list) una lista o array de pesos correspondientes a cada lag
    
    Return
        df_smoothed (dataframe)
    '''
    window_size = len(weights)
    weights = np.array(weights)
    
    # Extraer los valores de la columna de datos
    values = df.iloc[:, 0].values
    
    # Calcular el promedio móvil ponderado utilizando rolling y dot
    rolling_weights = df.rolling(window_size).apply(lambda x: np.dot(x, weights))
    df_smoothed = rolling_weights / sum(weights)

    # dropna
    df_smoothed = df_smoothed.dropna()
    return df_smoothed

In [13]:
# definir los pesos - en el ejemplo ventana movil de tamaño 5
weights = [0.05, 0.10, 0.15, 0.3, 0.4]
data_weighted_moving_average = apply_weighted_moving_average(data_sampled.copy(), weights)

In [14]:
fig_weighted_moving_average = plot_compare_tendencias(df_original = data_sampled, 
                                             df_smoothed = data_weighted_moving_average,
                                                      kind_smooth = 'weighted moving average - weights: [0.05, 0.10, 0.15, 0.3, 0.4]'
                                            )

# save
fig_weighted_moving_average.write_html("output_eda/2_univariate_analysis/fig_weighted_moving_average.html")

## Promedio Móvil Exponencial
- Los promedios móviles anteriores (simple o pesado) tienen un alcance finito en el tiempo: “olvidan” el pasado más allá de cierta distancia temporal.

- Este promedio asigna un peso que decrece exponencialmente, pero nunca se vuelve cero, a los valores pasado. Esto se define un factor de decaimiento alfa entre 0 y 1. Este factor de decamiento es cada vez más pequeño entre más lejos esta la observación

S3 = alfa*X3 + alfa(1-alfa)*X2 + (1-alfa)(1-alfa)*X1

In [15]:
def apply_exponential_moving_average(df, alpha):
    """
    Moving average
    
    Args
        df (dataframe)
        alpha (int)
    
    Return
        df_smoothed (dataframe)
    """
    df_smoothed = df.ewm(adjust = False,alpha = alpha).mean()
    df_smoothed = df_smoothed.dropna()

    return df_smoothed

In [16]:
# parametro alpha
alpha = 0.1
data_exponential_moving_average = apply_exponential_moving_average(data_sampled.copy(), alpha)

In [17]:
fig_exponential_moving_average = plot_compare_tendencias(df_original = data_sampled, 
                                             df_smoothed = data_exponential_moving_average,
                                                         kind_smooth = 'exponential moving average - alpha 0.1'
                                            )

# save
fig_exponential_moving_average.write_html("output_eda/2_univariate_analysis/fig_exponential_moving_average.html")